In [35]:
import os
import fnmatch
import threading
from typing import List, Optional

In [37]:
class SearchTimeoutError(Exception):
    """Custom exception raised when a search exceeds the maximum allowed time."""
    pass


class FileSearcher:

    _eager: bool = True

    @property
    def eager(self):
        return self._eager

    @eager.setter
    def eager(self, value):
        if isinstance(value, bool):
            self._eager = value

    def __init__(
        self,
        start_dir: Optional[str] = None,
        max_search_time: Optional[int] = None,
        eager: Optional[bool] = None
    ):
        """
        Initialize the FileSearcher class.

        Parameters
        ----------
        start_dir : str, optional
            The directory to start the search from.
            If not provided, the current working directory is used.
        max_search_time : int, optional
            The maximum amount of time in seconds allowed for the search.
            If the search exceeds this time, a `SearchTimeoutError` is raised.
        eager : Optional[bool]
            When set to `True`, the search stops when a single file is found.
        """
        self.start_dir = start_dir if start_dir else os.getcwd()
        self.max_search_time = max_search_time
        self.eager = eager
        self._timeout_event = threading.Event()

    def _start_timer(self):
        if self.max_search_time:
            timer = threading.Timer(self.max_search_time, self._raise_timeout)
            timer.start()
            self._timer = timer

    def _raise_timeout(self):
        self._timeout_event.set()
        raise SearchTimeoutError(f"Search exceeded the maximum allowed time of {self.max_search_time} seconds.")

    def _check_timeout(self):
        if self._timeout_event.is_set():
            raise SearchTimeoutError(f"Search exceeded the maximum allowed time of {self.max_search_time} seconds.")

    def search(self, filename: str, recursive: bool = True) -> List[str]:
        """
        Search for a file by its name starting from the start directory.

        Parameters
        ----------
        filename : str
            The name of the file to search for.
        recursive : bool, optional
            Whether to search recursively or not. Default is True.

        Returns
        -------
        List[str]
            A list of file paths that match the filename.
        """
        self._start_timer()
        matches = []

        if not filename:
            raise ValueError("Filename must not be empty.")

        if not os.path.isdir(self.start_dir):
            raise ValueError(f"Start directory '{self.start_dir}' is not a valid directory.")

        if recursive:
            for root, _, files in os.walk(self.start_dir):
                self._check_timeout()
                for name in files:
                    if fnmatch.fnmatch(name, filename):
                        matches.append(os.path.join(root, name))
        else:
            for name in os.listdir(self.start_dir):
                self._check_timeout()
                if fnmatch.fnmatch(name, filename):
                    matches.append(os.path.join(self.start_dir, name))

        self._timer.cancel()
        return matches

    def search_in_parents(self, filename: str, max_depth: int = 5) -> Optional[str]:
        """
        Search for a file by its name in parent directories up to a specified depth.

        Parameters
        ----------
        filename : str
            The name of the file to search for.
        max_depth : int, default=5
            The maximum number of parent directories to search in.

        Returns
        -------
        Optional[str]
            The first matching path found, or None if no match is found.
        """
        self._start_timer()
        current_dir = self.start_dir

        for _ in range(max_depth):
            self._check_timeout()
            potential_path = os.path.join(current_dir, filename)
            if os.path.isfile(potential_path):
                self._timer.cancel()
                return potential_path

            new_dir = os.path.dirname(current_dir)
            if new_dir == current_dir:  # Reached the root directory
                break
            current_dir = new_dir

        self._timer.cancel()
        return None

    def global_search(
        self, filename: str, eager: Optional[bool] = None
    ) -> List[str]:
        """
        Perform a global search for a file across the entire file system.

        Parameters
        ----------
        filename : str
            The name of the file to search for.
        eager : Optional[bool]
            When set to `True`, the search stops when a single file is found.

        Returns
        -------
        List[str]
            A list of file paths that match the filename.
        """
        self.eager = eager
        self._start_timer()
        matches = []

        for root, _, files in os.walk(os.path.sep):
            self._check_timeout()
            for name in files:
                if fnmatch.fnmatch(name, filename):
                    matches.append(os.path.join(root, name))
                    if self.eager:
                        self._timer.cancel()
                        return matches

        self._timer.cancel()
        return matches

    def intelligent_search(
        self,
        filename: str,
        max_parent_depth: int = 5,
        eager: Optional[bool] = None,
    ) -> List[str]:
        """
        Perform an intelligent search that first attempts to find the file locally,
        then in parent directories, and finally globally if necessary.

        Parameters
        ----------
        filename : str
            The name of the file to search for.
        max_parent_depth : int, default=5
            The maximum number of parent directories to search in.
        eager : Optional[bool]
            When set to `True`, the search stops when a single file is found.

        Returns
        -------
        List[str]
            A list of file paths that match the filename.
        """
        # Step 1: search in the current directory and its subdirectories
        try:
            local_results = self.search(filename)
            if local_results:
                return local_results
        except SearchTimeoutError:
            raise

        # Step 2: search in parent directories up to a certain depth
        try:
            parent_result = self.search_in_parents(filename, max_depth=max_parent_depth)
            if parent_result:
                return [parent_result]
        except SearchTimeoutError:
            raise

        # Step 3: perform a global search if no local or parent results are found
        try:
            global_results = self.global_search(filename, eager=eager)
            return global_results
        except SearchTimeoutError:
            raise


def find_file(
    filename: str,
    max_parent_depth: int = 5,
    eager: Optional[bool] = None,
    max_search_time: Optional[int] = None,
) -> List[str]:
    """
    Find a path for a file searching by its name.

    Parameters
    ----------
    filename : str
        The name of the file.
    max_parent_depth : int, default=5
        The maximum depth when searching through parents and subdirectories.
    eager : Optional[bool]
        When set to `True`, if the name is searched globally,
        then the search stops when a match is found.
    max_search_time : Optional[int]
        When specified, sets a maximum time limit for the search.

    Returns
    -------
    List[str]
        A list of strings representing the paths found for files matching
        the specified `filename`.
    """
    searcher = FileSearcher(max_search_time=max_search_time, eager=eager)
    return searcher.intelligent_search(filename, max_parent_depth=max_parent_depth)

In [38]:
searcher = FileSearcher(max_search_time=10)

# Search for a file named 'example.txt' starting from the current directory
results = searcher.intelligent_search('SelectNonCollinear.py')
print("Found files:", results)

Exception in thread Thread-8:
Traceback (most recent call last):
  File "/opt/anaconda3/envs/PyBlend/lib/python3.10/threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "/opt/anaconda3/envs/PyBlend/lib/python3.10/threading.py", line 1378, in run
    self.function(*self.args, **self.kwargs)
  File "/var/folders/t5/5_nvwwz94jq9nfwy6zv8s3_40000gn/T/ipykernel_3732/62494495.py", line 32, in _raise_timeout
__main__.SearchTimeoutError: Search exceeded the maximum allowed time of 10 seconds.


SearchTimeoutError: Search exceeded the maximum allowed time of 10 seconds.

In [39]:
searcher = FileSearcher(max_search_time=10)

# Search for a file named 'example.txt' starting from the current directory
results = searcher.intelligent_search('constructive.py')
print("Found files:", results)

Found files: ['/Users/erikingwersen/Desktop/github/PyBlend/pyblend/algorithm/constructive/constructive.py']


In [ ]:
# Search for 'config.yaml' in parent directories up to 3 levels
parent_search_result = searcher.search_in_parents('config.yaml', max_depth=3)
if parent_search_result:
    print(f"Found config file in parent directory: {parent_search_result}")
else:
    print("No config file found in parent directories.")